In [117]:
import random
import numpy as np 
import matplotlib.pyplot as plt 
import time
 

In [118]:
# Générer le graphe TSP incomplet
def generate_tsp_incomplete(num_sommets, max_poids_arete):
    matrice = [[None for _ in range(num_sommets)] for _ in range(num_sommets)]
    intervalles = {}
    for i in range(num_sommets):
        intervalle_start = random.randint(0, 10)
        intervalle_end = intervalle_start + random.randint(5, 20)
        intervalles[i] = (intervalle_start, intervalle_end)
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids
    for i in range(num_sommets):
        pas_connexe = True
        for j in range(num_sommets):
            if matrice[i][j] is None:
                matrice[i][j] = float('inf')
            else:
                pas_connexe = False
        if pas_connexe:
            poids = random.randint(1, max_poids_arete)
            matrice[i][num_sommets-1] = poids
            matrice[num_sommets-1][i] = poids
    return matrice, intervalles 

In [119]:
# Générer les données du problème
num_sommets = 5
max_poids_arete = 10
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete) 
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)  
# Afficher les intervalles 
print("\nIntervalles des sommets")
print(tsp_intervalles)   


Matrice pondérée:
[inf, 9, 5, inf, 2]
[9, inf, inf, 4, inf]
[5, inf, inf, 5, inf]
[inf, 4, 5, inf, 4]
[2, inf, inf, 4, inf]

Intervalles des sommets
{0: (2, 20), 1: (4, 16), 2: (1, 8), 3: (9, 21), 4: (3, 18)}


In [120]:
def calculate_path_length(matrice, solution):
    # Calculate the total length of the path
    length = 0
    for i in range(len(solution) - 1):
        length += matrice[solution[i]][solution[i + 1]]
    length += matrice[solution[-1]][solution[0]]  # return to the start point
    return length 


In [121]:
def is_valid_solution(solution, matrice, intervalles):
    # Validate if the solution respects the interval constraints
    path_length = calculate_path_length(matrice, solution)
    for city in range(len(solution)):
        min_weight, max_weight = intervalles[city]
        if not (min_weight <= path_length <= max_weight):
            return False
    return True 

In [122]:
def generate_neighbor_solution(current_solution, matrice, intervalles):
    # Generate a neighboring solution that respects the interval constraints
    attempts = 0
    while attempts < 100:  # Limit the number of attempts to avoid infinite loops
        neighbor = current_solution[:]
        i, j = random.sample(range(len(neighbor)), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        if is_valid_solution(neighbor, matrice, intervalles):
            return neighbor
        attempts += 1
    return current_solution 

In [123]:
def hill_climbing(matrice, intervalles, num_iterations=1000):
    num_sommets = len(matrice)
    execution_times = []
    distances = []
    
    # Generate an initial random valid solution
    while True:
        start_time = time.time()
        current_solution = list(range(num_sommets))
        random.shuffle(current_solution)
        if is_valid_solution(current_solution, matrice, intervalles):
            break
    current_length = calculate_path_length(matrice, current_solution)
    
    for _ in range(num_iterations):
        # Generate a valid neighboring solution
        next_solution = generate_neighbor_solution(current_solution, matrice, intervalles)
        next_length = calculate_path_length(matrice, next_solution)
        
        # If the neighboring solution is better, adopt it
        if next_length < current_length:
            current_solution = next_solution
            current_length = next_length
        
        end_time = time.time()
        execution_times.append(end_time - start_time)
        distances.append(current_length)  # Record the distance, not the solution
    
    return current_solution, current_length, execution_times, distances 
 

In [124]:
def plot_route(path, points):
    plt.figure(figsize=(10, 5))
    plt.scatter(points[:, 0], points[:, 1], c='blue')
    for i in range(len(path) - 1):
        from_node = path[i]
        to_node = path[i + 1]
        plt.plot([points[from_node, 0], points[to_node, 0]], [points[from_node, 1], points[to_node, 1]], 'k-')
    plt.title('Best Route Found')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()    

In [125]:
def plot_performance(execution_times, distances):
    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Execution Time (s)', color=color)
    ax1.plot(execution_times, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Best Path Length', color=color)  
    ax2.plot(distances, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()   
    plt.title('Performance Over Iterations')
    plt.show() 

In [126]:
points = np.random.rand(num_sommets, 2) * 100 
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row) 
best_solution, best_length, execution_times, distances = hill_climbing(tsp_matrice, tsp_intervalles) 
   
print(f"Solution trouvée : {best_solution} avec une longueur de chemin de : {best_length}")     
if best_solution:
    plot_route(best_solution, points)  

# plot_performance(execution_times, distances)   


Matrice pondérée:
[inf, 9, 5, inf, 2]
[9, inf, inf, 4, inf]
[5, inf, inf, 5, inf]
[inf, 4, 5, inf, 4]
[2, inf, inf, 4, inf]


KeyboardInterrupt: 

In [128]:
import random
import numpy as np
import matplotlib.pyplot as plt
import time
import pulp

# Générer le graphe TSP incomplet
def generate_tsp_incomplete(num_sommets, max_poids_arete):
    matrice = [[None for _ in range(num_sommets)] for _ in range(num_sommets)]
    intervalles = {}
    for i in range(num_sommets):
        intervalle_start = random.randint(0, 10)
        intervalle_end = intervalle_start + random.randint(5, 20)
        intervalles[i] = (intervalle_start, intervalle_end)
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids
    for i in range(num_sommets):
        pas_connexe = True
        for j in range(num_sommets):
            if matrice[i][j] is None:
                matrice[i][j] = float('inf')
            else:
                pas_connexe = False
        if pas_connexe:
            poids = random.randint(1, max_poids_arete)
            matrice[i][num_sommets-1] = poids
            matrice[num_sommets-1][i] = poids
    return matrice, intervalles

# Générer les données du problème
num_sommets = 5
max_poids_arete = 10
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete)
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

# Afficher les intervalles
print("\nIntervalles des sommets")
print(tsp_intervalles)

def calculate_path_length(matrice, solution):
    length = 0
    for i in range(len(solution) - 1):
        length += matrice[solution[i]][solution[i + 1]]
    length += matrice[solution[-1]][solution[0]]  # return to the start point
    return length

def is_valid_solution(solution, matrice, intervalles):
    path_length = calculate_path_length(matrice, solution)
    for city in solution:
        min_weight, max_weight = intervalles[city]
        if not (min_weight <= path_length <= max_weight):
            return False
    return True

def generate_neighbor_solution(current_solution, matrice, intervalles):
    # Generate a neighboring solution that respects the interval constraints
    for _ in range(100):  # Limit the number of attempts to avoid infinite loops
        neighbor = current_solution[:]
        i, j = random.sample(range(len(neighbor)), 2)
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        if is_valid_solution(neighbor, matrice, intervalles):
            return neighbor
    return current_solution  # Return the original solution if no valid neighbor is found

def hill_climbing(matrice, intervalles, num_iterations=1000):
    num_sommets = len(matrice)
    execution_times = []
    distances = []
    
    # Generate an initial random valid solution
    while True:
        start_time = time.time()
        current_solution = list(range(num_sommets))
        random.shuffle(current_solution)
        if is_valid_solution(current_solution, matrice, intervalles):
            break
    current_length = calculate_path_length(matrice, current_solution)
    
    for _ in range(num_iterations):
        # Generate a valid neighboring solution
        next_solution = generate_neighbor_solution(current_solution, matrice, intervalles)
        next_length = calculate_path_length(matrice, next_solution)
        
        # If the neighboring solution is better, adopt it
        if next_length < current_length:
            current_solution = next_solution
            current_length = next_length
        
        end_time = time.time()
        execution_times.append(end_time - start_time)
        distances.append(current_length)  # Record the distance, not the solution
    
    return current_solution, current_length, execution_times, distances

def plot_route(solution, points):
    route = points[solution]
    plt.figure(figsize=(10, 6))
    plt.plot(route[:, 0], route[:, 1], 'o-', label='Route')
    plt.plot([route[-1, 0], route[0, 0]], [route[-1, 1], route[0, 1]], 'o-')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('TSP Route')
    plt.legend()
    plt.show()

def plot_performance(execution_times, distances):
    plt.figure(figsize=(10, 6))
    plt.plot(distances, label='Path Length')
    plt.xlabel('Iteration')
    plt.ylabel('Path Length')
    plt.title('Hill Climbing Performance')
    plt.legend()
    plt.show()

points = np.random.rand(num_sommets, 2) * 100

print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)

best_solution, best_length, execution_times, distances = hill_climbing(tsp_matrice, tsp_intervalles)

print(f"Solution trouvée : {best_solution} avec une longueur de chemin de : {best_length}")

if best_solution:
    plot_route(best_solution, points)

plot_performance(execution_times, distances)



Matrice pondérée:
[inf, 2, 7, inf, 7]
[2, inf, 5, inf, 8]
[7, 5, inf, inf, 1]
[inf, inf, inf, inf, 10]
[7, 8, 1, 10, inf]

Intervalles des sommets
{0: (4, 18), 1: (10, 21), 2: (3, 20), 3: (5, 10), 4: (5, 22)}

Matrice pondérée:
[inf, 2, 7, inf, 7]
[2, inf, 5, inf, 8]
[7, 5, inf, inf, 1]
[inf, inf, inf, inf, 10]
[7, 8, 1, 10, inf]


KeyboardInterrupt: 